In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

In [3]:
!pip install --upgrade pip

In [4]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 4.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.6/285.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.2/372.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 MB 22.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 763.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 37.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [19]:
!pip install bert-extractive-summarizer

In [5]:
from autogluon.multimodal import MultiModalPredictor
from autogluon.core.utils.loaders import load_pd

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [42]:
path='../input/court-data/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')

In [43]:
train

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1
...,...,...,...,...,...
2473,TRAIN_2473,"HollyFrontier Cheyenne Refining, LLC, et al.","Renewable Fuels Association, et al.",Congress amended the Clean Air Act through the...,1
2474,TRAIN_2474,"Grupo Mexicano de Desarrollo, S. A.","Alliance Bond Fund, Inc.","Alliance Bond Fund, Inc., an investment fund, ...",1
2475,TRAIN_2475,Peguero,United States,"In 1992, the District Court sentenced Manuel D...",0
2476,TRAIN_2476,Immigration and Naturalization Service,St. Cyr,"On March 8, 1996, Enrico St. Cyr, a lawful per...",0


In [44]:
from summarizer import Summarizer

# BERT 모델 로드
model = Summarizer()

# `fact` 컬럼의 각 행을 요약 (train 데이터)
train['facts'] = train['facts'].apply(lambda x: ''.join(model(x, min_length=400)) if model(x, min_length=400) != '' else x)

# `fact` 컬럼의 각 행을 요약 (test 데이터)
test['facts'] = test['facts'].apply(lambda x: ''.join(model(x, min_length=400)) if model(x, min_length=400) != '' else x)


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:8

In [45]:
# 'fact' 컬럼의 문자열 길이에 대한 기본 통계를 계산합니다.
train_fact_lengths = train['facts'].apply(len)

# 'fact' 컬럼의 문자열 길이에 대한 평균, 최대, 최솟값을 계산합니다.
average_length = train_fact_lengths.mean()
max_length = train_fact_lengths.max()
min_length = train_fact_lengths.min()

print(f'Average length: {average_length}')
print(f'Maximum length: {max_length}')
print(f'Minimum length: {min_length}')

Average length: 1067.9673123486682
Maximum length: 5795
Minimum length: 15


In [46]:
import re
from nltk.tokenize import word_tokenize

# 텍스트 정제 함수
def clean_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)  # 특수 문자 제거
    text = text.lower()  # 모두 소문자로 변환
    text = text.strip()  # 불필요한 공백 제거
    return text

# 토큰화 함수
def tokenize_text(text):
    return word_tokenize(text)

# 정제 및 토큰화를 적용할 컬럼
columns_to_clean_and_tokenize = ['first_party', 'second_party', 'facts']

# 'train' 및 'test' 데이터셋의 각 컬럼에 함수 적용
for column in columns_to_clean_and_tokenize:
#     train[column] = train[column].apply(clean_text).apply(tokenize_text)
#     test[column] = test[column].apply(clean_text).apply(tokenize_text)
    train[column] = train[column].apply(clean_text)
    test[column] = test[column].apply(clean_text)

In [47]:
# 'fact' 컬럼의 문자열 길이에 대한 기본 통계를 계산합니다.
train_fact_lengths = train['facts'].apply(len)

# 'fact' 컬럼의 문자열 길이에 대한 평균, 최대, 최솟값을 계산합니다.
average_length = train_fact_lengths.mean()
max_length = train_fact_lengths.max()
min_length = train_fact_lengths.min()

print(f'Average length: {average_length}')
print(f'Maximum length: {max_length}')
print(f'Minimum length: {min_length}')

Average length: 1065.8256658595642
Maximum length: 5793
Minimum length: 13


In [48]:
train = train.drop('ID', axis=1)
test = test.drop('ID', axis=1)

In [49]:
train

,first_party,second_party,facts,first_party_winner
0,phil a st amant,herman a thompson,on june phil st amant a candidate ...,1
1,stephen duncan,lawrence owens,ramon nelson was riding his bike when he suffe...,0
2,billy joe magwood,tony patterson warden et al,an alabama state court convicted billy joe mag...,1
3,linkletter,walker,victor linkletter was convicted in state court...,0
4,william earl fikes,alabama,on april in selma alabama an intrud...,1
...,...,...,...,...
2473,hollyfrontier cheyenne refining llc et al,renewable fuels association et al,congress amended the clean air act through the...,1
2474,grupo mexicano de desarrollo s a,alliance bond fund inc,alliance bond fund inc an investment fund ...,1
2475,peguero,united states,in the district court sentenced manuel d...,0
2476,immigration and naturalization service,st cyr,on march enrico st cyr a lawful per...,0


In [ ]:
# predictor = MultiModalPredictor(label='first_party_winner',eval_metric='acc').fit(train_data=train,presets="best_quality",time_limit=3600*6)
predictor = MultiModalPredictor(label='first_party_winner',eval_metric='acc').fit(train_data=train,time_limit=1000)

No path specified. Models will be saved in: "AutogluonModels/ag-20230629_052357/"
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
AutoMM starts to create your model. ✨

- AutoGluon version is 0.8.0.

- Pytorch version is 1.13.1+cu117.

- Model will be saved to "/kaggle/working/AutogluonModels/ag-20230629_052357".

- Validation metric is "acc".

- To track the learning progress, you can open a terminal and launch Tensorboard:
    ```shell
    # Assume you have installed tensorboard
    tensorboard --logdir /kaggle/working/AutogluonModels/ag-20230629_052357
    ```

Enjoy your coffee, and let AutoMM do the job ☕☕☕ Learn more at https://auto.gluon.ai



/opt/conda/lib/python3.10/site-packages/autogluon/multimodal/utils/environment.py:59: UserWarning: Interactive environment is detected. Currently, MultiModalPredictor does not support multi-gpu training under an interactive environment due to the limitation of ddp / ddp_spawn strategies in PT Lightning. Thus, we switch to single gpu training. For multi-gpu training, you need to execute MultiModalPredictor in a script.
  warnings.warn(
2 GPUs are detected, and 1 GPUs will be used.
   - GPU 0 name: Tesla T4
   - GPU 0 memory: 11.90GB/15.84GB (Free/Total)
   - GPU 1 name: Tesla T4
   - GPU 1 memory: 15.74GB/15.84GB (Free/Total)
CUDA version is 11.7.



Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [ ]:
predictions = predictor.predict(test)

In [ ]:
predictions

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')
submission["first_party_winner"]=predictions
submission
submission.to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
zero_count = submission["first_party_winner"].value_counts()[0]
print("Number of zeroes in 'first_party_winner':", zero_count)